# 知识工程-作业10 英法中文翻译
2024214500 叶璨铭


## 代码与文档格式说明

> 本文档使用Jupyter Notebook编写，遵循Diátaxis 系统 Notebook实践 https://nbdev.fast.ai/tutorials/best_practices.html，所以同时包括了实验文档和实验代码。

> 本文档理论上支持多个格式，包括ipynb, docx, pdf 等。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。

> 为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对原本代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 代码规范说明

在我们实现函数过程中，函数的docstring应当遵循fastai规范而不是numpy规范，这样简洁清晰，不会Repeat yourself。相应的哲学和具体区别可以看 
https://nbdev.fast.ai/tutorials/best_practices.html#keep-docstrings-short-elaborate-in-separate-cells


为了让代码清晰规范，在作业开始前，使用 `ruff format`格式化助教老师给的代码; 

![alt text](image.png)


很好，这次代码格式化没有报错。

Pylance 似乎也没有明显问题。

## 实验环境准备

采用上次的作业专属环境，为了跑通最新方法，使用3.12 和 torch 2.7

```bash
conda create -n assignments python=3.12
conda activate assignments
pip install -r ../requirements.txt
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
pip install -U git+https://github.com/TorchRWKV/flash-linear-attention
```

本次作业似乎没有新的依赖，只是用到了 torch


## 原理回顾和课件复习



课上详细介绍了 概述、传统、统计、神经方法

概述中，注意到“平行语料库”这个概念，平行语料库（Parallel Corpus）是指在两个或多个语言之间具有对齐关系的文本集合。每对对应的文本在不同语言中表达了相同或相似的意义，因而可以用于语言间的比较、翻译模型的训练、语言学研究以及其他自然语言处理任务。罗塞塔石碑就是典型例子，罗塞塔石碑是一块刻有相同内容但用三种不同书写系统（古埃及象形文字、埃及民用文和古希腊文）的石碑。正是因为这些文本内容一致，学者们才能利用已知语言（古希腊文）的信息，逐步破译不明的古埃及象形文字，揭开了古埃及语言和文化的神秘面纱。

1947年，机器翻译认为是解读密码。1966年陷入低迷，1978年恢复。

难点是语言表达有歧义、文化有差异、翻译和知识、常识有关、解不唯一、新词和专有名词。

简单直接翻译方法直接替换已知的单词、短语、句子，然后调整顺序。

基于规则的方法用规则描述语法，对句子进行词法分析（把连续的字符序列划分成独立的词或符号（即“词元”或“标记”））、句法分析（在词法分析的基础上，利用预先定义的语法规则构造句子的句法结构（如语法树），确定不同词汇之间的组合规则和结构关系。）、语义分析（不仅关注词语的基本含义，还要判断它们在上下文中的语境作用，识别歧义、隐含意义和语义角色）。生成译文的句子结构（两个语言表达顺序不同），然后选择词法。
也叫作基于Transform的方法，这是独立分析两个语言的结构。缺点是语言写得不符合预定义的语法的时候处理不了。

注意，基于规则的方法无需依赖双语平行语料。（？需要词典，词典不算吗）

基于实例的方法，会类比已有的标准翻译实例，然后拼凑新的翻译。


基于统计的翻译，使用噪声信道模型。从S翻译到T，认为是T经过噪声干扰变成了S。求 P(T|S)， 
可以求 P(S|T) 和 P(T) 后者直接统计词频。










## 数据准备

助教已经帮我们preprocess好了数据，注意到有四个json文件。有两个是单词编码为id，词典对照。
train和valid是 jsonl 的格式， 一行是一个 句子翻译到一个句子。

![alt text](image-1.png)

## 补充完成`./metrics.py`中BLEU的计算

首先了解一下BLEU的概念, 参考课件和 https://en.wikipedia.org/wiki/BLEU 。

BLEU 全称 bilingual evaluation understudy，雙語替換評測，“Understudy”在这里指的是“替补演员”，在戏剧、表演等领域，指那些在正式演员因故无法出演时，能够随时顶替其角色的演员。

想要评测 Quality/correspondence/accuracy。 wiki说是2001, 课件是2002， IBM发明。不考虑可理解性、语法正确性，只考虑与参考答案（有一组， a set of good quality reference translations）像不像。输出[0,1]。数据集=语料库 corpus，有多个翻译预测和多组翻译参考答案。


首先需要指定 N-Gram 的 N，对于字符串，可以找到 其 N-Grams 的（不重复）集合。
![alt text](image-2.png)


然后定义出现次数C
![alt text](image-3.png)

S有M个预测答案，M组参考答案。
![alt text](image-4.png)

首先定义 Modified N-Gram Precision，既然是Precision，不是Recall，所以就是从预测答案来看

在预测的答案中，每一个N-Gram出现了很多次，对于每一个N-Gram而言，想看看在标准答案里面出现多少次，如果比我少，那我不准，我做多了，可能凭空翻译了新东西。有一组标准答案，所以里面对我最好的那个（出现我的ngram最多的那个）和我比。
![alt text](image-5.png)

这个式子另一个角度看，是为了衡量，参考句子多少个n gram时候是在 候选句子中有的，有多少次。

有了 Modified N-Gram Precision 之后，BLEU引入 Brevity penalty  简洁性惩罚（不是惩罚之后变简洁，而是简洁的被惩罚）。

因为刚才的指标不恰当地（unduly）会奖励那种为了拿分全部N-Gram都说一遍（telegraphic）的模型。

仔细看了看，我理解错了，惩罚的是有N-Gram，但是只说一遍，后面忘记说了的模型。


![alt text](image-6.png)

乍一看，
r是 real 的长度（一组里面最接近c的那个），c是candidate的长度。

如果c比r长就不惩罚了（我说的那个问题好像不是这里解决）

实际上，
需要注意BP是对整个语料库算的，不是对单个句子！是求了个和！整体进行惩罚！

![alt text](image-7.png)

![alt text](image-8.png)

最终，BLEU 认为 很多 N Gram都重要，要加权算，所以枚举n=1到无穷，有wn分布来加权，得到

![alt text](image-9.png)

几何平均数是希望，模型不是只在一个N Gram上表现好，而是大部分都好。

原本论文只考虑 n=1, 2, 3, 4, w = 1/4 。


 批评BLEU的意见指出，没有分词边界的语言，或者英语使用不同的token方案，会导致BLEU分数差异很大，不可比较。